In [1]:
from architectures_v1 import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, RNN
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

import os
import numpy as np
import tensorflow as tf
import pandas as pd
import time

## Loading Training/Testing/Validation Data

In [2]:
dataset = "PMNIST"

x_train_load = np.load(os.path.abspath(os.path.join('99_data', dataset,'x_train.npz')))
x_test_load = np.load(os.path.abspath(os.path.join('99_data', dataset, 'x_test.npz')))
x_valid_load = np.load(os.path.abspath(os.path.join('99_data', dataset,'x_valid.npz')))

x_train = x_train_load['arr_0']
x_test = x_test_load['arr_0']
x_valid = x_valid_load['arr_0']

y_train_load = np.load(os.path.abspath(os.path.join('99_data', dataset,'y_train.npz')))
y_test_load = np.load(os.path.abspath(os.path.join('99_data', dataset,'y_test.npz')))
y_valid_load = np.load(os.path.abspath(os.path.join('99_data', dataset,'y_valid.npz')))

y_train = y_train_load['arr_0']
y_test = y_test_load['arr_0']
y_valid = y_valid_load['arr_0']

print(f"x_train: {x_train.shape} - y_train: {y_train.shape}")
print(f"x_test: {x_test.shape} - y_test: {y_test.shape}")
print(f"x_valid: {x_valid.shape} - y_valid: {y_valid.shape}")

x_train: (50000, 784, 1) - y_train: (50000,)
x_test: (10000, 784, 1) - y_test: (10000,)
x_valid: (10000, 784, 1) - y_valid: (10000,)


In [3]:
seed = 42
tf.random.set_random_seed(seed)
np.random.seed(seed)

length = x_train.shape[1]
n_features = x_train.shape[-1]

model = Sequential()

model.add(RNN(LadderCell(units = 212,
                             max_delay = 782,
                             input_dims = 1), 
              input_shape = (length, n_features),
             return_sequences = False))
model.add(Dense(to_categorical(y_train).shape[-1], activation = "softmax"))

model.compile(optimizer = "adam", loss = "categorical_crossentropy", metrics = ["accuracy"])
model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rnn (RNN)                    (None, 212)               826022    
_________________________________________________________________
dense (Dense)                (None, 10)                2130      
Total params: 828,152
Trainable params: 213,495
Non-trainable params: 614,657
_________________________________________________________________


In [4]:
if not os.path.exists(os.path.abspath(os.path.join('models', dataset))):
    os.mkdir(os.path.abspath(os.path.join('models', dataset)))

file_path = os.path.abspath(os.path.join('models', dataset, f'{dataset}-ladder.hdf5'))

callbacks = [
    ModelCheckpoint(filepath=file_path, monitor='val_loss', save_best_only = True, mode = 'min', verbose = 1), 
    EarlyStopping(monitor = 'val_loss', patience = 10, mode = 'min')]

t = time.time()

result = model.fit(x_train, 
                   to_categorical(y_train),
                   epochs = 200, 
                   batch_size = 100, 
                   validation_data = (x_valid, to_categorical(y_valid)), 
                   callbacks = callbacks)

print(f"Training time: {time.time() - t} s")

df_results = pd.DataFrame(result.history)
df_results.to_csv(os.path.abspath(os.path.join('models', dataset, 'ladder_results.csv')))

Train on 50000 samples, validate on 10000 samples
Epoch 1/200
49900/50000 [============================>.] - ETA: 0s - loss: 0.2918 - acc: 0.9124
Epoch 00001: val_loss improved from inf to 0.17747, saving model to C:\Users\Sergio\Documents\GitHub_Repositories\01_Maintained\NeuralODE\models\PMNIST\PMNIST-laguerre.hdf5
50000/50000 [==============================] - 198s 4ms/sample - loss: 0.2917 - acc: 0.9124 - val_loss: 0.1775 - val_acc: 0.9464
Epoch 2/200
49900/50000 [============================>.] - ETA: 0s - loss: 0.1386 - acc: 0.9593
Epoch 00002: val_loss improved from 0.17747 to 0.14212, saving model to C:\Users\Sergio\Documents\GitHub_Repositories\01_Maintained\NeuralODE\models\PMNIST\PMNIST-laguerre.hdf5
50000/50000 [==============================] - 198s 4ms/sample - loss: 0.1387 - acc: 0.9593 - val_loss: 0.1421 - val_acc: 0.9572
Epoch 3/200
49900/50000 [============================>.] - ETA: 0s - loss: 0.1037 - acc: 0.9685
Epoch 00003: val_loss improved from 0.14212 to 0.12710

In [7]:
model.load_weights(file_path)
model.evaluate(x_test, to_categorical(y_test))

10000/10000 [==============================] - 37s 4ms/sample - loss: 0.0927 - acc: 0.9729


[0.0927379258789122, 0.9729]